In [1]:
import torch 
import optuna
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
import mlflow
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AdamW, get_linear_schedule_with_warmup
import rouge
from ipynb.fs.full.sari import SARIsent
import pandas as pd
from rouge import Rouge
from tqdm.auto import trange
import random
import numpy as np
import gc
import tensorflow as tf
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import seaborn as sns
import plotly.express as px
import pickle 
import mlflow
import mlflow.sklearn
import mlflow.tensorflow
import json
import os
import tempfile
from mlflow.models.signature import infer_signature
from sklearn.utils import shuffle
from torch import nn
import pickle

C:\Users\nkhozin\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('data\\train_small_medium_mix_clean.csv', index_col=0)
val = pd.read_csv('data\\eval.csv', index_col=0)

In [7]:
train_pairs = list(zip(train.source, train.target))

val = val[val['size']=='small']
eval_pairs = list(zip(val.source, val.target))

print(train.shape[0], val.shape[0])

50157 2560


In [8]:
class Params(object):
    def __init__(self, batch_size, test_batch_size, epochs, lr, momentum, seed, cuda, log_interval):
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.momentum = momentum
        self.seed = seed
        self.cuda = cuda
        self.log_interval = log_interval

In [9]:
experiment_name = "rut5_optimize_clear_data"
mlflow.set_experiment(experiment_name)
current_experiment = dict(mlflow.get_experiment_by_name(experiment_name))
exp_id = current_experiment['experiment_id']

In [12]:
def objective(trial):
    # Определение гиперпараметров
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 1000)
    num_layers = trial.suggest_int("num_layers", 2, 12)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    epochs = trial.suggest_int('epochs', 2, 5)
    
    batch_size = 3
    #epochs = 2
    
    model_name = 'cointegrated/rut5-base-multitask'
    model = T5ForConditionalGeneration.from_pretrained(model_name, num_layers=num_layers, dropout_rate=dropout_rate).cuda()
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    
    # Определение оптимизатора и планировщика
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=len(train_pairs) * epochs)
    
    # Training loop
    with mlflow.start_run(experiment_id=exp_id, run_name='first_test'):
        mlflow.log_param("model_name", 'rut5_multitask')
        mlflow.log_param("batch_size", learning_rate)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("learning_rate", batch_size)
        mlflow.log_param("weight_decay", weight_decay)
        mlflow.log_param("warmup_steps", warmup_steps)
        mlflow.log_param("num_layers", num_layers)
        mlflow.log_param("dropout_rate", dropout_rate)
        for epoch in range(epochs):
            model.train()
            train_loss = 0

            for i in trange(0, int(len(train_pairs) / batch_size)):
                try:
                    batch = train_pairs[i * batch_size: (i + 1) * batch_size]
                    x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
                    y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
                    y.input_ids[y.input_ids == 0] = -100

                except OutOfMemoryError:
                    print('Ignoring batch due to CUDA out of memory')
                    continue

                optimizer.zero_grad()
                loss = model(
                    input_ids=x.input_ids,
                    attention_mask=x.attention_mask,
                    labels=y.input_ids,
                    decoder_attention_mask=y.attention_mask,
                    return_dict=True
                ).loss

                train_loss += loss.data.item()
                loss.backward()
                optimizer.step()
                scheduler.step()
                torch.cuda.empty_cache()
                gc.collect()

            train_loss /= int(len(train_pairs) / batch_size)
            mlflow.log_metric("train_loss", train_loss, epoch)
            print(f'Epoch {epoch}, train_loss: {train_loss}')

            # Validation loop
            eval_loss = 0
            model.eval()
            with torch.no_grad():
                for j in trange(0, int(len(eval_pairs) / batch_size)):
                    batch = eval_pairs[j * batch_size: (j + 1) * batch_size]
                    x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
                    y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)

                    loss = model(
                        input_ids=x.input_ids,
                        attention_mask=x.attention_mask,
                        labels=y.input_ids,
                        decoder_attention_mask=y.attention_mask,
                        return_dict=True
                    ).loss

                    eval_loss += loss.data.item()

            eval_loss /= int(len(eval_pairs) / batch_size)
            mlflow.log_metric("eval_loss", eval_loss, epoch)
            print(f'Epoch {epoch}, eval_loss: {eval_loss}')
        return eval_loss

In [ ]:
study = optuna.create_study(direction='minimize')

# Запуск оптимизации гиперпараметров
study.optimize(objective, n_trials=35)

mlflow.end_run()

[I 2023-06-15 09:18:42,046] A new study created in memory with name: no-name-7c8f1570-c737-4d80-a475-89d7a86606b0
Some weights of the model checkpoint at cointegrated/rut5-base-multitask were not used when initializing T5ForConditionalGeneration: ['encoder.block.6.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.9.layer.1.layer_norm.weight', 'encoder.block.8.layer.1.DenseReluDense.wo.weight', 'encoder.block.7.layer.0.SelfAttention.k.weight', 'encoder.block.8.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.5.layer.1.layer_norm.weight', 'encoder.block.5.layer.0.SelfAttention.q.weight', 'encoder.block.4.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.10.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.6.layer.1.layer_norm.weight', 'encoder.block.10.layer.1.layer_norm.weight', 'encoder.block.11.layer.0.layer_norm.weight', 'encoder.block.5.layer.0.SelfAttention.o.weight', 'encoder.block.9.layer.0.SelfAttention.k.weight', 'encoder.block.4.layer.1.DenseReluDense.wo.weight', 'enc

Epoch 0, train_loss: 13.910052919995307


100%|██████████| 853/853 [00:12<00:00, 69.74it/s]


Epoch 0, eval_loss: 9.769611346344316


100%|██████████| 16719/16719 [55:29<00:00,  5.02it/s] 


...

100%|██████████| 16719/16719 [1:01:08<00:00,  4.56it/s]


Epoch 2, train_loss: 1.4947506007240134


100%|██████████| 853/853 [00:14<00:00, 60.62it/s]


Epoch 2, eval_loss: 6.779201609029021


 15%|█▍        | 2505/16719 [09:30<51:38,  4.59it/s]  

In [ ]:
# Получение результатов
best_trial = study.best_trial

print("Best trial:")
print("Value: ", best_trial.value)
print("Params: ")
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

In [ ]:
filehandler = open("optuna_trials_clean_data.obj","wb")
pickle.dump(study.get_trials(), filehandler)
filehandler.close()

In [4]:
with open('optuna_trials.obj', 'rb') as f:
    obj = pickle.load(f)

In [8]:
len(obj)

35

In [40]:
tb = []

for i in range(len(obj)):
    trial = obj[i]   
    params = {}
    params['iter'] = trial.number
    params.update(trial.params)
    params['loss'] = trial.values[0]
    tb.append(params)

In [43]:
pd.DataFrame(tb).drop('number', axis=1)

,iter,learning_rate,weight_decay,warmup_steps,num_layers,dropout_rate,loss
0,0,0.000098,0.105954,177,7,0.338209,2.641003
1,1,0.000055,0.164190,523,4,0.126000,1.450552
2,2,0.000037,0.077263,824,8,0.221739,1.581771
3,3,0.000045,0.145387,908,11,0.100246,1.190233
4,4,0.000025,0.113149,118,10,0.222361,1.508529
5,5,0.000028,0.101248,477,6,0.144912,1.364144
6,6,0.000016,0.196126,386,12,0.484830,2.387404
7,7,0.000011,0.043701,367,9,0.225881,1.513977
8,8,0.000036,0.146940,477,2,0.305286,6.200557
9,9,0.000032,0.064082,730,10,0.287230,1.703320


In [34]:
pd.DataFrame.from_dict(params, orient='index')

,0
number,0.000000
learning_rate,0.000098
weight_decay,0.105954
warmup_steps,177.000000
num_layers,7.000000
dropout_rate,0.338209
loss,2.641003
